In [12]:
import os
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as pl

from read_gsheet import get_specific_range


In [ ]:
get_specific_range('unpreparing_stady_data')

In [134]:
df=pd.read_csv('unpreparing_stady_data.csv',header=1, date_format=True, dayfirst=True).drop_duplicates()
df

,Ранг,Спринт,Модули\Спринты,Дата,Утро,Вечер,Доп занятия,Название
0,1,Спринт 1,Спринт 1,26.05.2025,125.0,109.0,NaN,NaN
1,1,Спринт 1,NaN,27.05.2025,124.0,125.0,NaN,NaN
2,1,Спринт 1,NaN,28.05.2025,103.0,NaN,NaN,NaN
3,1,Спринт 1,NaN,29.05.2025,NaN,NaN,NaN,NaN
4,1,Спринт 1,NaN,30.05.2025,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
376,#8,SQL Lab,NaN,NaN,NaN,NaN,NaN,NaN
377,#7,Python Lab,NaN,24.08.2025,NaN,60.0,NaN,NaN
378,#7,Python Lab,NaN,25.08.2025,120.0,11.0,NaN,NaN
379,#6,YandexGPT,NaN,24.08.2025,60.0,60.0,NaN,NaN


In [131]:
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 379 entries, 0 to 380
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ранг            230 non-null    object 
 1   Спринт          230 non-null    object 
 2   Модули\Спринты  28 non-null     object 
 3   Дата            376 non-null    object 
 4   Утро            43 non-null     float64
 5   Вечер           38 non-null     float64
 6   Доп занятия     24 non-null     float64
dtypes: float64(3), object(4)
memory usage: 23.7+ KB


In [136]:
df=df.rename(columns={'Ранг':'rank',
                   'Спринт':'activity_subtype_name',
                    'Дата':'activity_date',
                    'Утро':'morning',
                    'Вечер':'evening',
                    'Доп занятия':'add_study_duration',
                    'Название':'add_study_type'}).drop(columns=['Модули\Спринты','rank'])

In [137]:
df.activity_date=pd.to_datetime(df['activity_date'])

C:\Users\Misterius\AppData\Local\Temp\ipykernel_5004\687405557.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.activity_date=pd.to_datetime(df['activity_date'])


In [138]:
df=df.fillna(0)

### Перевод данных из полей morning, evening и add_study в отдельные строки. Приведение к формату duration - daypart  

In [139]:
df.query('morning != 0')

,activity_subtype_name,activity_date,morning,evening,add_study_duration,add_study_type
0,Спринт 1,2025-05-26 00:00:00,125.0,109.0,0.0,0
1,Спринт 1,2025-05-27 00:00:00,124.0,125.0,0.0,0
2,Спринт 1,2025-05-28 00:00:00,103.0,0.0,0.0,0
10,Спринт 1,2025-06-05 00:00:00,77.0,0.0,0.0,Вебинар
13,Спринт 1,2025-06-08 00:00:00,15.0,0.0,0.0,0
15,Спринт 2,2025-06-10 00:00:00,112.0,0.0,0.0,0
16,Спринт 2,2025-06-11 00:00:00,129.0,95.0,0.0,0
17,Спринт 2,2025-06-12 00:00:00,118.0,123.0,0.0,0
28,Спринт 3,2025-06-23 00:00:00,133.0,0.0,0.0,0
29,Спринт 3,2025-06-24 00:00:00,93.0,130.0,0.0,0
